# Lab 3 - Data Formatting Pipeline

**Notebook:** `01_data_formatting_pipeline.ipynb`  
**Objective:** We aim to implement the **first data pipeline** from the Data Management Backbone - from raw data in the Landing Zone to cleaned and standardized datasets in the Formatted Zone, using PySpark.

### Group: *L3-T04*

#### Group Members: **Marvin Ernst, Oriol Gelabert, Alex Malo**

Class: **23D020 - Big Data Management for Data Science**

Date: *June 23, 2025*

In this notebook we include the following two parts of the project (from section A):
- A.1: Explore and understand the raw datasets
- A.2: Clean and format the data using Spark


---

### Steps in this pipeline:
1. **Data Ingestion and Exploration**  
   Load example raw files of each dataset, perform schema checks, and preview content.

2. **Data Cleaning and Standardization**  
   - Handle inconsistent delimiters, missing values, and encoding issues  
   - Harmonize schemas across years and rename columns  
   - Normalize text and parse dates, coordinates, and identifiers

3. **Formatted Output**  
   Save cleaned datasets in Parquet format to the `formatted_zone/`, ready for further transformation.

---


**We selected the three following datasets:**
1. Tourist Housing
2. Household Size (JSON)
3. Commercial Premises

---

## **A.1 - Explore the data and choose the KPI’s**

#### Setup: Load Libraries and Define Paths

In [ ]:
from pathlib import Path
import os
import zipfile
import re
import warnings
warnings.simplefilter("ignore", category=FutureWarning)

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import udf, col, to_date, concat, lit, substring, when, regexp_replace, sum as _sum, round, count, create_map, lower, trim
from pyspark.sql.types import StringType
from unidecode import unidecode

#### Initialize Spark

In [ ]:
appName = "EDA"
master = "local[*]" # Spark will use all cores (*) available
if not 'sc' in globals(): # This 'trick' makes sure the SparkContext sc is initialized exactly once
  conf = SparkConf().setAppName(appName).setMaster(master).set("spark.driver.memory", "4g")
  sc = SparkContext(conf=conf)
spark = SparkSession(sc)

25/06/24 02:38:42 WARN Utils: Your hostname, MacBook-Pro-40.local resolves to a loopback address: 127.0.0.1; using 192.168.1.23 instead (on interface en0)
25/06/24 02:38:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/24 02:38:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/24 02:38:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/24 02:38:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/24 02:38:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50890)
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/anaconda3/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/anaconda3/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/anaconda3/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/Users/Admin/Library/Caches/pypoetry/virtualenvs/lab3-spark-project-pYxmljys-py3.12/lib/python3.12/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/Users/Admin/Library/Caches/pypoetry/virtualenvs/lab3-spark-project-pYxmljys-py3.12/lib/python3.12/site-packages/pyspark/accumulators.

#### Define Project Paths
This ensures our notebook works regardless of the system.

In [3]:
project_root = Path().resolve().parent
landing = project_root / "landing_zone"
formatted = project_root / "formatted_zone"
formatted.mkdir(parents=True, exist_ok=True)

#### Unzip Raw Files in the Landing Zone
We ensure all `.zip` files are extracted and ready for reading.

In [4]:
for file in landing.glob("*.zip"):
    try:
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(landing)
            print(f"Extracted: {file.name} to {landing}")
        file.unlink()  
        print(f"Deleted ZIP: {file.name}")
    except zipfile.BadZipFile:
        print(f"Skipped (not a valid ZIP): {file.name}")

#### Load Raw Datasets
Recall we have three different datasets:
- csv with tourist housing information
- json with household size (number of people living in a house)
- csv with with ground floor premises for economic activity

Let's first open them:

In [5]:
print("Available files in landing zone:")
for file in landing.iterdir():
    print("-", file.name)

Available files in landing zone:
- 2022_pad_dom_mdbas_n-persones.json
- 2024_3T_hut_comunicacio_opendata.csv
- 2019_4T_hut_comunicacio.csv
- 2022_1T_hut_comunicacio.csv
- 2019_pad_dom_mdbas_n-persones.json
- 2022_4T_hut_comunicacio.csv
- 2019_1T_hut_comunicacio.csv
- 2024_4T_hut_comunicacio_opendata.csv
- 220930_censcomercialbcn_opendata_2022_v10_mod.csv
- 2021_1T_hut_comunicacio.csv
- 2021_4T_hut_comunicacio.csv
- 2020_4T_hut_comunicacio.csv
- 2020_1T_hut_comunicacio.csv
- 2023_pad_dom_mdbas_n-persones.json
- 2024_2T_hut_comunicacio_opendata.csv
- 241021_censcomercialbcn_opendata_2024_v5.csv
- 2024_pad_dom_mdbas_n-persones.json
- 2023_1T_hut_comunicacio.csv
- 2023_4T_hut_comunicacio_opendata.csv
- 2021_3T_hut_comunicacio.csv
- 2019_3T_hut_comunicacio.csv
- 2022_3T_hut_comunicacio.csv
- 2023_3T_hut_comunicacio_opendata.csv
- 2020_2T_hut_comunicacio.csv
- 2020_pad_dom_mdbas_n-persones.json
- 2019_censcomercialbcn_detall.csv
- 2021_2T_hut_comunicacio.csv
- 2020_3T_hut_comunicacio.csv
- 2

We pick a file from each dataset:

In [6]:
df_csv = spark.read.option("header", True).csv(f"{landing}/2019_1T_hut_comunicacio.csv")
df_household=  spark.read.option("multiline", True).json(f"{landing}/2022_pad_dom_mdbas_n-persones.json")
df_comerce = spark.read.option("header", True).csv(f"{landing}/220930_censcomercialbcn_opendata_2022_v10_mod.csv")

### Exploratory Data Analysis (EDA)

Now, we visualize the picked file of each type. 

For each selected dataset, we:
- View the schema with `.printSchema()`
- Display a few rows with `.show()`
- Use `.describe()` and `.count()` to summarize numerical fields
- Identify missing values or irregular entries if any

#### **Tourist Housing**

In [7]:
df_csv.show(5)
df_csv.printSchema()
df_csv.count()
df_csv.describe().show()
df_csv.columns

+------------+------------+--------------------+------------+---------------+---------+----+-------+----+-------+----+------+------+---+-----+-----------+-----------+
| N_EXPEDIENT|   DISTRICTE|               BARRI|TIPUS_CARRER|         CARRER|TIPUS_NUM|NUM1|LLETRA1|NUM2|LLETRA2|BLOC|PORTAL|ESCALA|PIS|PORTA| LONGITUD_X|  LATITUD_Y|
+------------+------------+--------------------+------------+---------------+---------+----+-------+----+-------+----+------+------+---+-----+-----------+-----------+
|01-2013-0753|CIUTAT VELLA|            el Raval|      CARRER|       HOSPITAL|        1|  25|   NULL|NULL|   NULL|NULL|  NULL|  NULL|  1|    2|2.172227285|41.38088335|
|02-2014-0566|  L'EIXAMPLE|  la Sagrada Família|    AVINGUDA|          GAUDI|        1|  51|   NULL|  53|   NULL|NULL|  NULL|  NULL| EN|    1|2.174215606|41.40844312|
|02-2014-0207|  L'EIXAMPLE|la Dreta de l'Eix...|       PLAÇA|         TETUAN|        1|  33|   NULL|NULL|   NULL|NULL|  NULL|  NULL| SA|    2|2.176060643| 41.3956364

25/06/24 02:38:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------+-------------------+---------------+------------+----------+------------------+------------------+-------+------------------+-------+------------------+------+------------------+------------------+------------------+--------------------+--------------------+
|summary| N_EXPEDIENT|          DISTRICTE|          BARRI|TIPUS_CARRER|    CARRER|         TIPUS_NUM|              NUM1|LLETRA1|              NUM2|LLETRA2|              BLOC|PORTAL|            ESCALA|               PIS|             PORTA|          LONGITUD_X|           LATITUD_Y|
+-------+------------+-------------------+---------------+------------+----------+------------------+------------------+-------+------------------+-------+------------------+------+------------------+------------------+------------------+--------------------+--------------------+
|  count|        9561|               9561|           9551|        9561|      9561|              9561|              9561|    128|               791|      3|  

['N_EXPEDIENT',
 'DISTRICTE',
 'BARRI',
 'TIPUS_CARRER',
 'CARRER',
 'TIPUS_NUM',
 'NUM1',
 'LLETRA1',
 'NUM2',
 'LLETRA2',
 'BLOC',
 'PORTAL',
 'ESCALA',
 'PIS',
 'PORTA',
 'LONGITUD_X',
 'LATITUD_Y']

The most important insight is that these files lack district and neighbourhood codes. However, given that they have the names, we can generate a dictionary to map district codes to names, and then parse these files through.

#### **Household by Size**

In [8]:
df_household.show(5)
df_household.printSchema()
df_household.count()
df_household.describe().show()
df_household.columns

+---+----------+--------------+---------------+--------------+---------+-------------+-------------+-----+
|AEB|Codi_Barri|Codi_Districte|Data_Referencia|N_PERSONES_AGG|Nom_Barri|Nom_Districte|Seccio_Censal|Valor|
+---+----------+--------------+---------------+--------------+---------+-------------+-------------+-----+
|  1|         1|             1|     2022-01-01|             1| el Raval| Ciutat Vella|         1001|  200|
|  1|         1|             1|     2022-01-01|             2| el Raval| Ciutat Vella|         1001|  127|
|  1|         1|             1|     2022-01-01|             3| el Raval| Ciutat Vella|         1001|   78|
|  1|         1|             1|     2022-01-01|             4| el Raval| Ciutat Vella|         1001|   54|
|  1|         1|             1|     2022-01-01|             5| el Raval| Ciutat Vella|         1001|   20|
+---+----------+--------------+---------------+--------------+---------+-------------+-------------+-----+
only showing top 5 rows

root
 |-- AE

['AEB',
 'Codi_Barri',
 'Codi_Districte',
 'Data_Referencia',
 'N_PERSONES_AGG',
 'Nom_Barri',
 'Nom_Districte',
 'Seccio_Censal',
 'Valor']

As we can see above, column "valor" has information on how many houselds have "N_PERSONES_AGG" people living, for each neighbourhood "Codi_Barri". This JSON file does not have any messy nested structure, and spark seems to have no trouble reading the file (ie. there are no column name mismatches). 

#### **Comercial Premises**

In [9]:
df_comerce.show(5)
df_comerce.printSchema()
df_comerce.count()
df_comerce.describe().show()
df_comerce.columns

+--------------------+-----------+------------------------+-----------------------+---------------------+--------------------+-------------------+------------------+-------------------+--------------------+-------------------+--------------------+--------------+------------+--------------------+-----------+----------+---------+---------+----------+----------+-----------+-------------+--------------+------+-------+------------------+-----------------+------------+-----------+--------------------+--------+--------+------+-----+-------------------+--------------+-----------------+------------+------+------------+---------+-------------+----------+-----------+--------------+-------------+--------------------+------------+
|           ID_Global|ID_Bcn_2016|Codi_Principal_Activitat|Nom_Principal_Activitat|Codi_Sector_Activitat|Nom_Sector_Activitat|Codi_Grup_Activitat|Nom_Grup_Activitat|Codi_Activitat_2022|       Nom_Activitat|Codi_Activitat_2016|           Nom_Local|SN_Oci_Nocturn|SN_Coworki

+-------+--------------------+------------------+------------------------+-----------------------+---------------------+--------------------+-------------------+--------------------+-------------------+--------------------+-------------------+--------------------+--------------+------------+--------------------+-----------+----------+---------+---------+---------------+----------+-----------+-------------+----------------+------+------------+------------------+------------------+--------------------+-------------------+--------------------+------------------+--------------+------+------------------+-------------------+--------------+------------------+------------+------------------+------------------+------------------+------------------+------------------+---------------+------------------+-------------------+--------------------+------------+
|summary|           ID_Global|       ID_Bcn_2016|Codi_Principal_Activitat|Nom_Principal_Activitat|Codi_Sector_Activitat|Nom_Sector_Activitat|C

['ID_Global',
 'ID_Bcn_2016',
 'Codi_Principal_Activitat',
 'Nom_Principal_Activitat',
 'Codi_Sector_Activitat',
 'Nom_Sector_Activitat',
 'Codi_Grup_Activitat',
 'Nom_Grup_Activitat',
 'Codi_Activitat_2022',
 'Nom_Activitat',
 'Codi_Activitat_2016',
 'Nom_Local',
 'SN_Oci_Nocturn',
 'SN_Coworking',
 'SN_Servei_Degustacio',
 'SN_Obert24h',
 'SN_Mixtura',
 'SN_Carrer',
 'SN_Mercat',
 'Nom_Mercat',
 'SN_Galeria',
 'Nom_Galeria',
 'SN_CComercial',
 'Nom_CComercial',
 'SN_Eix',
 'Nom_Eix',
 'X_UTM_ETRS89',
 'Y_UTM_ETRS89',
 'Latitud',
 'Longitud',
 'Direccio_Unica',
 'Codi_Via',
 'Nom_Via',
 'Planta',
 'Porta',
 'Num_Policia_Inicial',
 'Lletra_Inicial',
 'Num_Policia_Final',
 'Lletra_Final',
 'Solar',
 'Codi_Parcela',
 'Codi_Illa',
 'Seccio_Censal',
 'Codi_Barri',
 'Nom_Barri',
 'Codi_Districte',
 'Nom_Districte',
 'Referencia_Cadastral',
 'Data_Revisio']

The data appears well-structured and does not suffer from nested structures or mismatches. However, we notice that many fields are stored as **string**, including numeric fields (e.g., `Codi_Activitat`, `X_UTM_ETRS89`), and some columns may have **"Sí"/"No"** values (e.g. `SN_Coworking`) that need to be **converted to binary indicators** (`1`/`0`). Further, a few location-related columns like `Nom_Mercat` or `Nom_Galeria` contain a large number of nulls, which is expected due to sparsity.

This dataset will be standardized, relevant columns cast to the proper types, and written to the **Formatted Zone** for later use in KPI analysis.

##### **Comparing two versions of the commercial premises dataset:**

Additionally to the one from 2021–2022 (df_comerce) and we also show one from 2019 (df_comerce_19).

In [10]:
df_comerce.show(5)
df_comerce_19 = spark.read.option("header", True).csv(f"{landing}/2019_censcomercialbcn_detall.csv")
df_comerce_19.show(5)

+--------------------+-----------+------------------------+-----------------------+---------------------+--------------------+-------------------+------------------+-------------------+--------------------+-------------------+--------------------+--------------+------------+--------------------+-----------+----------+---------+---------+----------+----------+-----------+-------------+--------------+------+-------+------------------+-----------------+------------+-----------+--------------------+--------+--------+------+-----+-------------------+--------------+-----------------+------------+------+------------+---------+-------------+----------+-----------+--------------+-------------+--------------------+------------+
|           ID_Global|ID_Bcn_2016|Codi_Principal_Activitat|Nom_Principal_Activitat|Codi_Sector_Activitat|Nom_Sector_Activitat|Codi_Grup_Activitat|Nom_Grup_Activitat|Codi_Activitat_2022|       Nom_Activitat|Codi_Activitat_2016|           Nom_Local|SN_Oci_Nocturn|SN_Coworki

This shows slight differences in column names, missing values, and completeness, for example, ID_Global vs. ID_Bcn_2019, or lowercase vs. uppercase naming. These discrepancies imply that we need to standardize the schemas before integrating or analyzing data across years.

## **A.2 - Data Formatting Process**

### Transformations - Standardize and Clean Data

This section covers all transformations applied to raw datasets, including renaming inconsistent column names, casting column types, fixing delimiters or headers, and harmonizing location fields (like coordinates or districts). These steps are crucial for aligning data from different sources and years into a unified schema.

The first action we take is to standardize all column types, as different years have varying types for several columns.

#### **Comercial Premises**

##### *Column types:*
- **ID_Global** -> *string*  
- **ID_Bcn_2016** -> *int*  
- **Codi_Principal_Activitat** -> *int*  
- **Nom_Principal_Activitat** -> *string*  
- **Codi_Sector_Activitat** -> *int*  
- **Nom_Sector_Activitat** -> *string*  
- **Codi_Grup_Activitat** -> *int*  
- **Nom_Grup_Activitat** -> *string*  
- **Codi_Activitat_2022** -> *int*  
- **Nom_Activitat** -> *string*  
- **Codi_Activitat_2016** -> *int*  
- **Nom_Local** -> *string*  
- **SN_Oci_Nocturn** -> *boolean*  
- **SN_Coworking** -> *boolean*  
- **SN_Servei_Degustacio** -> *boolean*  
- **SN_Obert24h** -> *boolean*  
- **SN_Mixtura** -> *boolean*  
- **SN_Carrer** -> *boolean*  
- **SN_Mercat** -> *boolean*  
- **Nom_Mercat** -> *string*  
- **SN_Galeria** -> *boolean*  
- **Nom_Galeria** -> *string*  
- **SN_CComercial** -> *boolean*  
- **Nom_CComercial** -> *string*  
- **SN_Eix** -> *boolean*  
- **Nom_Eix** -> *string*  
- **X_UTM_ETRS89** -> *float*  
- **Y_UTM_ETRS89** -> *float*  
- **Latitud** -> *float*  
- **Longitud** -> *float*  
- **Direccio_Unica** -> *string*  
- **Codi_Via** -> *int*  
- **Nom_Via** -> *string*  
- **Planta** -> *string*  
- **Porta** -> *int*  
- **Num_Policia_Inicial** -> *int*  
- **Lletra_Inicial** -> *string*  
- **Num_Policia_Final** -> *int*  
- **Lletra_Final** -> *string*  
- **Solar** -> *int*  
- **Codi_Parcela** -> *int*  
- **Codi_Illa** -> *int*  
- **Seccio_Censal** -> *int*  
- **Codi_Barri** -> *int*  
- **Nom_Barri** -> *string*  
- **Codi_Districte** -> *int*  
- **Nom_Districte** -> *string*  
- **Referencia_Cadastral** -> *string*  
- **Data_Revisio** -> *datetime*  

#### **Tourist Housing**

##### *Column types:*
- **N_EXPEDIENT** -> *string*  
- **DISTRICTE** -> *string*  
- **BARRI** -> *string*  
- **TIPUS_CARRER** -> *string*  
- **CARRER** -> *string*  
- **TIPUS_NUM** -> *int*  
- **NUM1** -> *int*  
- **LLETRA1** -> *string*  
- **NUM2** -> *int*  
- **LLETRA2** -> *string*  
- **BLOC** -> *string*  
- **PORTAL** -> *string*  
- **ESCALA** -> *string*  
- **PIS** -> *string*  
- **PORTA** -> *string*  
- **LONGITUD_X** -> *float*  
- **LATITUD_Y** -> *float*  

Additionally, given that the column **N_EXPEDIENT** encodes the registration year, we can extract this information to generate a new `year` column for temporal analysis.

#### **Household Size**

##### *Column types:*
- **AEB** -> *int*  
- **Codi_Barri** -> *int*  
- **Codi_Districte** -> *int*  
- **Data_Referencia** -> *date*  
- **N_PERSONES_AGG** -> *int*  
- **Nom_Barri** -> *string*  
- **Nom_Districte** -> *string*  
- **Seccio_Censal** -> *int*  
- **Valor** -> *int*  

### Build Location Mapping Dictionaries

We load a sample tourist housing dataset to extract reference mappings for district and neighborhood codes. These will later be used to enrich other files that do not contain explicit code columns.

In [11]:
df_tourist_23 = spark.read.option("header", True).csv(f"{landing}/2023_1T_hut_comunicacio.csv")
district_dict = {
    row["DISTRICTE"]: row["CODI_DISTRICTE"]
    for row in df_tourist_23.select("CODI_DISTRICTE", "DISTRICTE").distinct().collect()
}

print(f"District Dictionary: {district_dict}")

# Build Neighbourhood Mapping:
neighborhood_dict = {
    row["BARRI"]: row["CODI_BARRI"]
    for row in df_tourist_23.select( "CODI_BARRI", "BARRI").distinct().collect()
}

print(f"Neighborhood Dictionary: {neighborhood_dict}")


# Flatten the dictionary into a list of alternating key, value for create_map:
district_map_expr = create_map(
    *[item for k, v in district_dict.items() for item in (lit(k), lit(v))]
)

neighbourhood_map_expr = create_map(
    *[item for k, v in neighborhood_dict.items() for item in (lit(k), lit(v))]
)

District Dictionary: {'SANTS-MONTJUÏC': '3', "L'EIXAMPLE": '2', 'GRACIA': '6', 'HORTA-GUINARDÓ': '7', 'LES CORTS': '4', 'NOU BARRIS': '8', 'SANT ANDREU': '9', 'CIUTAT VELLA': '1', 'SANT MARTI': '10', 'SARRIA-SANT GERVASI': '5'}
Neighborhood Dictionary: {"la Nova Esquerra de l'Eixample": '9', 'la Guineueta': '48', 'Sant Gervasi - Galvany': '26', 'la Teixonera': '38', 'la Marina del Prat Vermell': '12', 'Sant Antoni': '10', 'Sant Genís dels Agudells': '39', 'Sant Andreu': '60', 'les Corts': '19', "la Vall d'Hebron": '41', 'el Bon Pastor': '59', 'Verdun': '51', 'la Verneda i la Pau': '73', 'el Poble Sec': '11', 'la Bordeta': '16', 'el Barri Gòtic': '2', 'Horta': '43', 'Diagonal Mar i el Front Marítim del Poblenou': '69', 'les Roquetes': '50', "la Dreta de l'Eixample": '7', 'Porta': '45', 'Canyelles': '49', 'la Sagrada Família': '6', "el Camp d'en Grassot i Gràcia Nova": '32', 'el Congrés i els Indians': '62', 'Sants - Badal': '17', 'el Raval': '1', 'Sant Martí de Provençals': '72', 'Vilap

#### Define Accent-Stripping UDF

We define a custom UDF using `unidecode` to normalize district and neighborhood names by removing accents and converting to lowercase. This improves matching across datasets.

In [12]:
unidecode_udf = udf(lambda x: unidecode(x.lower()) if x is not None else None, StringType())

### Load and Clean HUT Datasets (Tourist Housing)

We iterate through all HUT CSV files using a year–quarter pattern. Files are manually corrected (e.g. encoding, delimiters, misnamed headers) and harmonized to a standard schema.  
This section includes:
- Encoding fixes (`iso-8859-1` vs `UTF-8`)
- Column renaming and standardization
- Type casting (e.g. coordinates, identifiers)
- Date extraction from expedition ID

2023-3T needed manual correction   
2023-1T needed manual correction  
2020-3T header mispecified , has to be taken form 2020-4T    
2021-3T some rows with ; as separators, manually corrected

In [13]:
# We match files using the pattern "YYYY_QT", e.g., "2020_1T":
pattern = re.compile(r"(20\d{2})_(\dT)", re.IGNORECASE)

# We define a dictionary to store the cleaned DataFrames for each year and quarter:
df_dict = {}

# Then we iterate through all files in the landing zone:
for file in os.listdir(landing):
    match = pattern.match(file)
    if match:
        year = match.group(1)
        quarter = match.group(2)
        df_name = f"hut_{year}_{quarter}"
        full_path = os.path.join(landing, file)

        input_path = full_path
        temp_path = full_path + ".tmp"

        # For files from early 2020, we use ISO encoding and fix delimiters and location names:
        if year == '2020' and (quarter in ['1T', '2T']):
            with open(input_path, "r", encoding="iso-8859-1") as f_in, open(temp_path, "w", encoding="iso-8859-1") as f_out:
                for i, line in enumerate(f_in):
                    line_replaced = line.replace(";", ",")
                    line_replaced = line_replaced.replace(",Sant Pere, Santa Caterina i la Ribera", ',"Sant Pere, Santa Caterina i la Ribera"')
                    line_final = line_replaced.replace(",Vallvidrera, el Tibidabo i les Planes", ',"Vallvidrera, el Tibidabo i les Planes"')
                    f_out.write(line_final)
        else:
            # For other files, we apply UTF-8 and fix common formatting issues in 2023:
            with open(input_path, "r", encoding="UTF-8") as f_in, open(temp_path, "w", encoding="UTF-8") as f_out:
                for i, line in enumerate(f_in):
                    if i == 0 and year == '2023' and (quarter == '1T' or quarter == '3T'):
                        line = line.replace("LATITUD_Y", "LATITUD_Y\n")
                    line_replaced = line.replace(";", ",")
                    line_replaced = line_replaced.replace(",Sant Pere, Santa Caterina i la Ribera", ',"Sant Pere, Santa Caterina i la Ribera"')
                    line_final = line_replaced.replace(",Vallvidrera, el Tibidabo i les Planes", ',"Vallvidrera, el Tibidabo i les Planes"')
                    f_out.write(line_final)

        # We replace the original file with the cleaned temporary version:
        os.replace(temp_path, input_path)

        try:
            # We load the cleaned CSV file using the correct encoding:
            if year == '2020' and (quarter in ['1T', '2T']):
                df = spark.read.option("header", True).option("encoding", "iso-8859-1").csv(full_path)
            else:
                df = spark.read.option("header", True).option("encoding", "UTF-8").csv(full_path)

            # We rename inconsistent column names across files:
            df = df.withColumnsRenamed({
                'BARRI': 'DES_BARRI',
                'TIPUS_CARRER': 'COD_TIPUS_CARRER',
                'CARRER': 'DES_CARRER',
                'LLETRA1': 'DES_LLETRA_1',
                'LLETRA2': 'DES_LLETRA_2',
                'BLOC': 'DES_BLOC',
                'PORTAL': 'DES_PORTAL',
                'ESCALA': 'DES_ESCALA',
                'PIS': 'DES_PIS',
                'PORTA': 'DES_PORTA',
                'NOM_BARRI': 'DES_BARRI',
                'NOM_DISTRICTE': 'DES_DISTRICTE',
                'DISTRICTE': 'DES_DISTRICTE',
                'NUMERO_REGISTRE_GENERALITAT': 'NUM_REGISTRE',
                'NUMERO_PLACES': 'NUM_PLACES',
                'N_EXPEDIENT': 'ID_EXPEDIENT'
            })

            # We cast the street number fields to integers and drop the originals:
            df = df.withColumn("COD_TIPUS_NUM", col("TIPUS_NUM").cast("int")) \
                   .withColumn("NUM_CARRER_1", col("NUM1").cast("int")) \
                   .withColumn("NUM_CARRER_2", col("NUM2").cast("int")) \
                   .drop("NUM1", "NUM2", "TIPUS_NUM")

            # If coordinate columns exist, we convert them to float and fix decimal format:
            if "LONGITUD_X" in df.columns and "LATITUD_Y" in df.columns:
                df = df.withColumn("NUM_LONGITUD_X", regexp_replace(col("LONGITUD_X"), ",", ".").cast("float")) \
                       .withColumn("NUM_LATITUD_Y", regexp_replace(col("LATITUD_Y"), ",", ".").cast("float")) \
                       .drop("LONGITUD_X", "LATITUD_Y")

            # If district code is missing, we assign it using the district name dictionary:
            if "CODI_DISTRICTE" not in df.columns:
                df = df.withColumn("COD_DISTRICTE", district_map_expr.getItem(col("DES_DISTRICTE")))

            # If both codes are present, we cast them to integer and remove originals:
            if "CODI_BARRI" in df.columns:
                df = df.withColumn("COD_BARRI", col("CODI_BARRI").cast("int")) \
                       .withColumn("COD_DISTRICTE", col("CODI_DISTRICTE").cast("int")) \
                       .drop("CODI_BARRI", "CODI_DISTRICTE")

            # We extract the registration date from the expedition ID:
            df = df.withColumn("DAT_ALTA_EXPEDIENT", to_date(concat(lit("01-"), substring(col("ID_EXPEDIENT"), 1, 7)), "dd-MM-yyyy"))

            # We normalize the district names by removing accents:
            if "DES_DISTRICTE" in df.columns:
                df = df.withColumn("DES_DISTRICTE", unidecode_udf("DES_DISTRICTE")) \
                       .drop("DISTRICTE")

            # We save the cleaned DataFrame to the formatted zone:
            try:
                df.write.parquet(f"{formatted}/hut_{year}_{quarter}.parquet", mode='overwrite')
            except Exception as e:
                print(f"Failed to write parquet for year {year} and quarter {quarter}: {e}")

            # We store the cleaned DataFrame in a dictionary for future access:
            df_dict[df_name] = df
            print(f"Loaded: {df_name} from {file}")

        except Exception as e:
            print(f"Error loading {file}: {e}")

Loaded: hut_2024_3T from 2024_3T_hut_comunicacio_opendata.csv
Loaded: hut_2019_4T from 2019_4T_hut_comunicacio.csv
Loaded: hut_2022_1T from 2022_1T_hut_comunicacio.csv
Loaded: hut_2022_4T from 2022_4T_hut_comunicacio.csv
Loaded: hut_2019_1T from 2019_1T_hut_comunicacio.csv
Loaded: hut_2024_4T from 2024_4T_hut_comunicacio_opendata.csv
Loaded: hut_2021_1T from 2021_1T_hut_comunicacio.csv
Loaded: hut_2021_4T from 2021_4T_hut_comunicacio.csv
Loaded: hut_2020_4T from 2020_4T_hut_comunicacio.csv
Loaded: hut_2020_1T from 2020_1T_hut_comunicacio.csv
Loaded: hut_2024_2T from 2024_2T_hut_comunicacio_opendata.csv
Loaded: hut_2023_1T from 2023_1T_hut_comunicacio.csv
Loaded: hut_2023_4T from 2023_4T_hut_comunicacio_opendata.csv
Loaded: hut_2021_3T from 2021_3T_hut_comunicacio.csv
Loaded: hut_2019_3T from 2019_3T_hut_comunicacio.csv
Loaded: hut_2022_3T from 2022_3T_hut_comunicacio.csv
Loaded: hut_2023_3T from 2023_3T_hut_comunicacio_opendata.csv
Loaded: hut_2020_2T from 2020_2T_hut_comunicacio.csv
L

#### Preview Cleaned HUT Dataset

We preview one of the cleaned tourist housing datasets to verify schema and transformations.

In [14]:
df_dict['hut_2019_3T'].show()

+------------+--------------+--------------------+----------------+-----------------+------------+------------+--------+----------+----------+-------+---------+-------------+------------+------------+--------------+-------------+-------------+------------------+
|ID_EXPEDIENT| DES_DISTRICTE|           DES_BARRI|COD_TIPUS_CARRER|       DES_CARRER|DES_LLETRA_1|DES_LLETRA_2|DES_BLOC|DES_PORTAL|DES_ESCALA|DES_PIS|DES_PORTA|COD_TIPUS_NUM|NUM_CARRER_1|NUM_CARRER_2|NUM_LONGITUD_X|NUM_LATITUD_Y|COD_DISTRICTE|DAT_ALTA_EXPEDIENT|
+------------+--------------+--------------------+----------------+-----------------+------------+------------+--------+----------+----------+-------+---------+-------------+------------+------------+--------------+-------------+-------------+------------------+
|01-2013-0753|  ciutat vella|            el Raval|          Carrer|         HOSPITAL|        NULL|        NULL|    NULL|      NULL|      NULL|      1|        2|            1|          25|        NULL|     2.1711

This confirms the desired changes.

### Load and Clean Household Size Data

We load household demographic data stored in JSON format. Each file corresponds to one year and contains neighborhood-level population counts.  
Steps include:
- Renaming fields
- Type casting
- Standardizing location and date fields

In [15]:
# First, we match files using the pattern "YYYY_pad_dom_mdbas_n":
pattern = re.compile(r"(20\d{2})_pad_dom_mdbas_n", re.IGNORECASE)

# We store each year's cleaned household dataset in a dictionary:
df_households = {}

# Then, again, we loop through files in the landing directory:
for file in os.listdir(landing):
    match = pattern.match(file)
    if match:
        year = match.group(1)
        df_name = f"pad_dom_{year}"
        full_path = os.path.join(landing, file)

        try:
            # We read the JSON file using multiline option to support nested structures:
            df = spark.read.option("multiline", True).json(full_path)

            # We rename the neighborhood column for consistency with other datasets:
            df = df.withColumnsRenamed({'Nom_Barri': 'DES_BARRI'})

            # We cast all relevant fields to appropriate data types and normalize the district name:
            df = df.withColumn("COD_AEB", col("AEB").cast("int")) \
                   .withColumn("DES_DISTRICTE", unidecode_udf('Nom_Districte')) \
                   .withColumn("COD_BARRI", col("Codi_Barri").cast("int")) \
                   .withColumn("COD_DISTRICTE", col("Codi_Districte").cast("int")) \
                   .withColumn("NUM_PERSONES_AGG", col("N_PERSONES_AGG").cast("int")) \
                   .withColumn("COD_SECCIO_CENSAL", col("Seccio_Censal").cast("int")) \
                   .withColumn("NUM_VALOR", col("Valor").cast("int")) \
                   .withColumn("DAT_REF", to_date(col("Data_Referencia"), "yyyy-MM-dd")) \
                   .drop(
                       'AEB', 'Codi_BARRI', 'Codi_DISTRICTE', 'Valor',
                       "Data_Referencia", "Seccio_Censal", 'N_PERSONES_AGG', 'Nom_Districte'
                   )

            # We write the cleaned file to the formatted zone in Parquet format:
            try:
                df.write.parquet(f"{formatted}/household_{year}_.parquet", mode='overwrite')
            except Exception as e:
                print(f"Failed to write parquet for year {year} : {e}")

            # We store the DataFrame in our dictionary:
            df_households[df_name] = df
            print(f"Loaded: {df_name} from {file}")
        except Exception as e:
            print(f"Error loading {file}: {e}")

Loaded: pad_dom_2022 from 2022_pad_dom_mdbas_n-persones.json
Loaded: pad_dom_2019 from 2019_pad_dom_mdbas_n-persones.json
Loaded: pad_dom_2023 from 2023_pad_dom_mdbas_n-persones.json
Loaded: pad_dom_2024 from 2024_pad_dom_mdbas_n-persones.json
Loaded: pad_dom_2020 from 2020_pad_dom_mdbas_n-persones.json
Loaded: pad_dom_2021 from 2021_pad_dom_mdbas_n-persones.json


#### Preview Cleaned Household Dataset

Here we inspect the first entries from a cleaned household size dataset:

In [16]:
df_households['pad_dom_2019'].show(5)

+---------+-------+-------------+---------+-------------+----------------+-----------------+---------+----------+
|DES_BARRI|COD_AEB|DES_DISTRICTE|COD_BARRI|COD_DISTRICTE|NUM_PERSONES_AGG|COD_SECCIO_CENSAL|NUM_VALOR|   DAT_REF|
+---------+-------+-------------+---------+-------------+----------------+-----------------+---------+----------+
| el Raval|      1| ciutat vella|        1|            1|               1|             1001|      206|2019-01-01|
| el Raval|      1| ciutat vella|        1|            1|               2|             1001|      118|2019-01-01|
| el Raval|      1| ciutat vella|        1|            1|               3|             1001|       75|2019-01-01|
| el Raval|      1| ciutat vella|        1|            1|               4|             1001|       44|2019-01-01|
| el Raval|      1| ciutat vella|        1|            1|               5|             1001|       24|2019-01-01|
+---------+-------+-------------+---------+-------------+----------------+--------------

This confirms the desired changes.

### Load and Clean Commercial Premises Data

We identify all CSV files containing censal commercial data by filename patterns. For each file, we:
- Handle schema changes between years (e.g. 2019 vs 2022)
- Normalize boolean flags and activity codes
- Standardize coordinate, location, and activity fields
- Write clean data to the formatted zone for further use

This is how the Coworking data is with Sí and No:

In [20]:
df = spark.read.option("header", True).csv(f"{landing}/220930_censcomercialbcn_opendata_2022_v10_mod.csv")
df.select("SN_Coworking", "SN_Oci_Nocturn").distinct().show(20, truncate=False)

+------------+--------------+
|SN_Coworking|SN_Oci_Nocturn|
+------------+--------------+
|No          |Sí            |
|No          |No            |
|Sí          |No            |
+------------+--------------+



In [21]:
# We define a pattern to detect commercial census CSVs containing the year (e.g., "censcomercialbcn_2022_..."):
pattern = re.compile(r"^(?=.*censcomercialbcn.*)(?=.*(20\d{2})_).*$", re.IGNORECASE)

# We define a dictionary to store cleaned DataFrames by year:
df_comercial = {}

# We define a helper function to robustly parse "Sí"/"Si"/"sí"/"yes"/"1" etc. into binary indicator values:
normalize_udf = udf(lambda x: unidecode(x.lower().strip()) if x else "", StringType())
def parse_boolean_indicator(column):
    return when(normalize_udf(col(column)).isin("si", "yes", "true", "1"), 1).otherwise(0)

# We iterate through all matching files in the landing directory:
for file in os.listdir(landing):
    match = pattern.match(file)
    if match:
        year = match.group(1)
        df_name = f"comercial_{year}"
        full_path = os.path.join(landing, file)

        try:
            # We read the CSV file with headers:
            df = spark.read.option("header", True).csv(full_path)

            # We normalize the date column depending on the version:
            if "ID_Bcn_2019" in df.columns:
                df = df.withColumnRenamed("ID_Bcn_2019", "ID_Global") \
                       .withColumn("DAT_REV", to_date(col("Data_Revisio"), "yyyyMMdd")) \
                       .drop("Data_Revisio")
            else:
                df = df.withColumn("DAT_REV", to_date(col("Data_Revisio"), "yyyy-MM-dd")) \
                       .drop("Data_Revisio")

            # We standardize the naming of the cadastral reference column:
            if "Referencia_cadastral" in df.columns:
                df = df.withColumnRenamed("Referencia_cadastral", "ID_REFERENCIA_CATASTRAL")
            else:
                df = df.withColumnRenamed("Referencia_Cadastral", "ID_REFERENCIA_CATASTRAL")

            # We normalize activity codes depending on year-specific naming:
            if "Codi_Activitat_2019" in df.columns:
                df = df.withColumn("COD_ACTIVITAT_19", col("Codi_Activitat_2019").cast("int")) \
                       .drop("Codi_Activitat_2019")
            else:
                df = df.withColumn("COD_ACTIVITAT_22", col("Codi_Activitat_2022").cast("int")) \
                       .drop("Codi_Activitat_2022")

            # We rename all descriptive and location-based columns to follow a consistent naming convention:
            df = df.withColumnsRenamed({
                'ID_Global': 'ID_GLOBAL',
                'Nom_Principal_Activitat': 'DES_ACTIVITAT_PRINCIPAL',
                'Nom_Sector_Activitat': 'DES_SECTOR',
                'Nom_Grup_Activitat': 'DES_GRUP',
                'Nom_Activitat': 'DES_ACTIVITAT',
                'Nom_Local': 'DES_LOCAL',
                'Nom_Mercat': 'DES_MERCAT',
                'Nom_Galeria': 'DES_GALERIA',
                'Nom_CComercial': 'DES_CENTRE_COMERCIAL',
                'Nom_Eix': 'DES_EIX_COMERCIAL',
                'Direccio_Unica': 'DES_DIRECCIO_UNICA',
                'Nom_Via': 'DES_VIA',
                'Planta': 'COD_PLANTA',
                'Lletra_Inicial': 'COD_LLETRA_INICIAL',
                'Lletra_Final': 'COD_LLETRA_FINAL',
                'Nom_Barri': 'DES_BARRI',
                'Codi_Activitat_2016': 'COD_ACTIVITAT_16',
                'Porta': 'COD_PORTA'
            })

            # We cast and clean district, group, and activity identifiers to integer types:
            df = df.withColumn("COD_ACTIVITAT_PRINCIPAL", col("Codi_Principal_Activitat").cast("int")) \
                   .drop("Codi_Principal_Activitat") \
                   .withColumn("DES_DISTRICTE", unidecode_udf("Nom_Districte")) \
                   .drop("Nom_Districte") \
                   .withColumn("ID_2016", col("ID_Bcn_2016").cast("int")) \
                   .drop("ID_Bcn_2016") \
                   .withColumn("COD_SECTOR", col("Codi_Sector_Activitat").cast("int")) \
                   .drop("Codi_Sector_Activitat") \
                   .withColumn("COD_GRUP", col("Codi_Grup_Activitat").cast("int")) \
                   .drop("Codi_Grup_Activitat")

            # We apply the helper function to convert boolean indicator columns into clean binary flags:
            df = df.withColumn("IND_OCI_NOCTURN", parse_boolean_indicator("SN_Oci_Nocturn")).drop("SN_Oci_Nocturn") \
                   .withColumn("IND_COWORKING", parse_boolean_indicator("SN_Coworking")).drop("SN_Coworking") \
                   .withColumn("IND_SERVEI_DEGUSTACIO", parse_boolean_indicator("SN_Servei_Degustacio")).drop("SN_Servei_Degustacio") \
                   .withColumn("IND_OBERT24H", parse_boolean_indicator("SN_Obert24h")).drop("SN_Obert24h") \
                   .withColumn("IND_MIXT", parse_boolean_indicator("SN_Mixtura")).drop("SN_Mixtura") \
                   .withColumn("IND_PEU_CARRER", parse_boolean_indicator("SN_Carrer")).drop("SN_Carrer") \
                   .withColumn("IND_MERCAT", parse_boolean_indicator("SN_Mercat")).drop("SN_Mercat") \
                   .withColumn("IND_GALERIA", parse_boolean_indicator("SN_Galeria")).drop("SN_Galeria") \
                   .withColumn("IND_CENTRE_COMERCIAL", parse_boolean_indicator("SN_CComercial")).drop("SN_CComercial") \
                   .withColumn("IND_EIX_COMERCIAL", parse_boolean_indicator("SN_Eix")).drop("SN_Eix")

            # We cast all coordinate and numeric location fields to float or int for correct processing:
            df = df.withColumn("NUM_X_UTM_ETRS89", col("X_UTM_ETRS89").cast("float")).drop("X_UTM_ETRS89") \
                   .withColumn("NUM_Y_UTM_ETRS89", col("Y_UTM_ETRS89").cast("float")).drop("Y_UTM_ETRS89") \
                   .withColumn("NUM_LATITUD", col("Latitud").cast("float")).drop("Latitud") \
                   .withColumn("NUM_LONGITUD", col("Longitud").cast("float")).drop("Longitud") \
                   .withColumn("COD_VIA", col("Codi_Via").cast("int")).drop("Codi_Via") \
                   .withColumn("NUM_POLICIA_INICIAL", col("Num_Policia_Inicial").cast("int")).drop("Num_Policia_Inicial") \
                   .withColumn("NUM_POLICIA_FINAL", col("Num_Policia_Final").cast("int")).drop("Num_Policia_Final") \
                   .withColumn("COD_SOLAR", col("Solar").cast("int")).drop("Solar") \
                   .withColumn("COD_PARCELA", col("Codi_Parcela").cast("int")).drop("Codi_Parcela") \
                   .withColumn("COD_ILLA", col("Codi_Illa").cast("int")).drop("Codi_Illa") \
                   .withColumn("COD_SECCIO_CENSAL", col("Seccio_Censal").cast("int")).drop("Seccio_Censal") \
                   .withColumn("COD_BARRI", col("Codi_Barri").cast("int")).drop("Codi_Barri") \
                   .withColumn("COD_DISTRICTE", col("Codi_Districte").cast("int")).drop("Codi_Districte")

            # We store the cleaned DataFrame in our dictionary:
            df_comercial[df_name] = df

            # We write the cleaned and normalized data to the formatted zone, partitioned by district and neighborhood:
            df.write.partitionBy("COD_DISTRICTE", "COD_BARRI") \
                   .parquet(f"{formatted}/comercial_{year}.parquet", mode="overwrite")

            print(f"Loaded and saved: {df_name}")

        except Exception as e:
            print(f"Error processing file {file}: {e}")

Loaded and saved: comercial_2022


Loaded and saved: comercial_2024


Loaded and saved: comercial_2019


Confirm the changes that we had troubles with:

In [ ]:
df = spark.read.parquet(f"{formatted}/comercial_2022.parquet")
df.groupBy("IND_COWORKING", "IND_OCI_NOCTURN").count().show()

+-------------+---------------+-----+
|IND_COWORKING|IND_OCI_NOCTURN|count|
+-------------+---------------+-----+
|            1|              0|  324|
|            0|              0|65545|
|            0|              1|  219|
+-------------+---------------+-----+



25/06/24 11:28:04 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1019432 ms exceeds timeout 120000 ms
25/06/24 11:28:04 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/24 11:28:05 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1240)
	at 